Sta 663 - Statistical Computing and Computation - Midterm 2
-----------
Due Wednesday, April 28th by 5:00 pm.



## Setup

In [2]:
# Load necessary packages
import torch
import numpy as np

## Task 1 - `newton()` function

Firstly, check whether the objective or derivatives are finite at the initial theta. Within each iteration (until maxit), calculate the jacobian and hessian of objective at the current theta and check whether they are finite. Then check whether current objective value meets convergence and calculate the current step size. Change the current theta by the step size computed. If the current objective value got larger, then half the step size until the objective value got smaller or the maximum iterations for halfing a step was met. 

In [32]:
def newton(theta, f, tol = 1e-8, fscale = 1.0,maxit = 100, max_half = 20):
    
    all_theta_i = [theta]
    all_f_i = [f(theta)]
    g_i = torch.autograd.functional.jacobian(f,theta)
    if not (torch.isfinite(f(theta)) and torch.isfinite(g_i).all()):
        raise ValueError("Objective function is not finite at the initial theta.")
    
    
    theta_i = theta
    
    for i in range(maxit):
        f_i = f(theta_i)
        flag_max_it = True
        g_i = torch.autograd.functional.jacobian(f,theta_i)
        h_i = torch.autograd.functional.hessian(f,theta_i)
        if not torch.isfinite(g_i).all():
            raise ValueError("Gradient is not finite at the initial theta.")
        if not torch.isfinite(h_i).all(): 
            raise ValueError("Hessian is not finite at the initial theta.")
        step = - np.linalg.solve(h_i, g_i)
        
        if max(abs(g_i)) < (abs(f_i)+fscale)*tol :
            flag_max_it = False
            break
        
        for j in range(max_half):
            flag_max_half = True
            new_theta_i = theta_i + step
            new_f_i = f(new_theta_i)
            if (new_f_i < all_f_i[-1]):
                flag_max_half = False
                break
            step /= 2
        
        if flag_max_half:
            raise ValueError("Max_half steps failed to reduce objective.")
        theta_i, f_i = new_theta_i, new_f_i
        all_theta_i.append(theta_i)
        all_f_i.append(f_i)
        
    
    
    if flag_max_it:
        raise ValueError("Max_it reached without convergence.")
        
    if not torch.linalg.cholesky_ex(h_i).info.eq(0):
        raise ValueError("Hessian is not positive finite at convergence.")
        
    result = {
        "f": all_f_i[-1],
        "theta": all_theta_i[-1],
        "iter": i + 1,
        "grad": g_i
    }
    
    return result

## Task 2 - Minimization examples


### 1. Quadratic function


In [5]:
## Objective function implementation
Qf = lambda x: x[0]**2 - 2 * x[0] + 2 * x[1]**2 + x[1] + 3 

In [6]:
## theta 1 minimization
newton(torch.tensor([1.5,1],dtype = torch.double), Qf, tol = 1e-8, fscale = 1.0,maxit = 100, max_half = 20)

{'f': tensor(1.8750, dtype=torch.float64),
 'theta': tensor([ 1.0000, -0.2500], dtype=torch.float64),
 'iter': 2,
 'grad': tensor([0., 0.], dtype=torch.float64)}

In [7]:
## theta 2 minimization
newton(torch.tensor([0.01,0.1],dtype = torch.double), Qf, tol = 1e-8, fscale = 1.0,maxit = 100, max_half = 20)

{'f': tensor(1.8750, dtype=torch.float64),
 'theta': tensor([ 1.0000, -0.2500], dtype=torch.float64),
 'iter': 2,
 'grad': tensor([0.0000e+00, 1.1102e-16], dtype=torch.float64)}

In [8]:
## theta 3 minimization
newton(torch.tensor([10.1,10.1],dtype = torch.double), Qf, tol = 1e-8, fscale = 1.0,maxit = 100, max_half = 20)

{'f': tensor(1.8750, dtype=torch.float64),
 'theta': tensor([ 1.0000, -0.2500], dtype=torch.float64),
 'iter': 2,
 'grad': tensor([0., 0.], dtype=torch.float64)}

### 2. Sphere function in 10d

In [9]:
## Objective function implementation
Sf = lambda x: x[0]**2 + x[1]**2 + x[2]**2+ x[3]**2+ x[4]**2+ x[5]**2+ x[6]**2+ x[7]**2+ x[8]**2+ x[9]**2

In [10]:
## theta 1 minimization
newton(torch.tensor([1,1,1,1,1,1,1,1,1,1],dtype = torch.double), Sf, tol = 1e-8, fscale = 1.0,maxit = 100, max_half = 20)

{'f': tensor(0., dtype=torch.float64),
 'theta': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64),
 'iter': 2,
 'grad': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64)}

In [11]:
## theta 2 minimization
newton(torch.tensor([1,100,10000,1,10,10,-1,10,1,10],dtype = torch.double), Sf, fscale = 1.0,tol = 1e-8, maxit = 100, max_half = 20)

{'f': tensor(0., dtype=torch.float64),
 'theta': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64),
 'iter': 2,
 'grad': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64)}

In [12]:
## theta 3 minimization
newton(torch.tensor([-1,-1,10000,1000000,1000,10000,1000,-100,1000,1000],dtype = torch.double), Sf, tol = 1e-8, fscale = 1.0,maxit = 100, max_half = 20)

{'f': tensor(0., dtype=torch.float64),
 'theta': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64),
 'iter': 2,
 'grad': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64)}

### 3. Rosenbrock's function

In [13]:
## Objective function implementation
Rf = lambda x: 100 * (x[1] - x[0]**2)**2 + (1-x[0])**2

In [14]:
## theta 1 minimization
newton(torch.tensor([0.01,0.1],dtype = torch.double), Rf, tol = 1e-8, fscale = 1.0,maxit = 100, max_half = 20)

{'f': tensor(1.1093e-31, dtype=torch.float64),
 'theta': tensor([1.0000, 1.0000], dtype=torch.float64),
 'iter': 16,
 'grad': tensor([-6.6613e-16,  0.0000e+00], dtype=torch.float64)}

In [15]:
## theta 2 minimization
newton(torch.tensor([-10,-100],dtype = torch.double), Rf, tol = 1e-8,fscale = 1.0, maxit = 100, max_half = 20)

{'f': tensor(1.4787e-20, dtype=torch.float64),
 'theta': tensor([1.0000, 1.0000], dtype=torch.float64),
 'iter': 53,
 'grad': tensor([ 4.2357e-09, -2.1725e-09], dtype=torch.float64)}

In [16]:
## theta 3 minimization
newton(torch.tensor([2,2],dtype = torch.double), Rf, tol = 1e-8, fscale = 1.0,maxit = 100, max_half = 20)

{'f': tensor(1.2331e-28, dtype=torch.float64),
 'theta': tensor([1.0000, 1.0000], dtype=torch.float64),
 'iter': 16,
 'grad': tensor([ 1.1058e-13, -4.4409e-14], dtype=torch.float64)}

### 4. Rosenbrock’s function in 4d

#### 4d Rosenbrock's function might have more than one minima so that Newton fails with inappropriate starting point.

In [17]:
## Objective function implementation
Rf_4d = lambda x: 100 * (x[1] - x[0]**2)**2 + (1-x[0])**2 + 100 * (x[2] - x[1]**2)**2 + (1-x[1])**2 + 100 * (x[3] - x[2]**2)**2 + (1-x[2])**2

In [18]:
## theta 1 minimization
newton(torch.tensor([2,2,2,2],dtype = torch.double), Rf_4d, tol = 1e-8, fscale = 1.0,maxit = 100, max_half = 20)

{'f': tensor(6.7748e-28, dtype=torch.float64),
 'theta': tensor([1.0000, 1.0000, 1.0000, 1.0000], dtype=torch.float64),
 'iter': 21,
 'grad': tensor([ 1.8696e-13, -7.1054e-14,  6.5725e-13, -3.1086e-13],
        dtype=torch.float64)}

In [19]:
## theta 2 minimization
newton(torch.tensor([-2,-2,-2,-2],dtype = torch.double), Rf_4d, tol = 1e-8,fscale = 1.0, maxit = 100, max_half = 20)

ValueError: Step failed to reduce the objective after max_half step halvings.

In [20]:
## theta 3 minimization
newton(torch.tensor([-2,-2,-2,2],dtype = torch.double), Rf_4d, tol = 1e-8,fscale = 1.0, maxit = 100, max_half = 20)

{'f': tensor(5.9868e-27, dtype=torch.float64),
 'theta': tensor([1.0000, 1.0000, 1.0000, 1.0000], dtype=torch.float64),
 'iter': 24,
 'grad': tensor([ 1.1324e-14,  8.9040e-14,  2.2160e-13, -2.2204e-13],
        dtype=torch.float64)}

### 5. Beale function

#### The Beale function is nonconvex and multimodal, with sharp peaks at the corners of the input domain. The point newton method is trying to find might be the saddle point and convergence is not assured.

In [21]:
## Objective function implementation
Bf = lambda x : (1.5 - x[0] + x[0] * x[1])**2 + (2.25 - x[0] + x[0]*x[1]**2)**2 + (2.625 - x[0] + x[0]*x[1]**3)**2

In [22]:
## theta 1 minimization
newton(torch.tensor([-1,-1],dtype = torch.double), Bf, tol = 1e-8,fscale = 1.0, maxit = 100, max_half = 20)

ValueError: Step failed to reduce the objective after max_half step halvings.

In [23]:
## theta 2 minimization
newton(torch.tensor([2,1],dtype = torch.double), Bf, tol = 1e-8, fscale = 1.0,maxit = 100, max_half = 20)

ValueError: Step failed to reduce the objective after max_half step halvings.

In [24]:
## theta 3 minimization
newton(torch.tensor([2,0],dtype = torch.double), Bf, tol = 1e-8,fscale = 1.0, maxit = 100, max_half = 20)

{'f': tensor(1.3960e-27, dtype=torch.float64),
 'theta': tensor([3.0000, 0.5000], dtype=torch.float64),
 'iter': 7,
 'grad': tensor([-4.1467e-14,  2.5280e-13], dtype=torch.float64)}


### 6. Poisson regression likelihood

$$ \begin{aligned} \log \lambda_i &= \beta_0 + \beta_1 x_i \ L(\beta_0,\beta_1) &= \prod_{i=1}^{10}\frac{\lambda_i^{y_i} e^{-\lambda_i}}{y_i!} \ l(\beta_0,\beta_1) &= \sum_{i=1}^{10} y_i \log \lambda_i - \lambda_i - \log y_i! \end{aligned} $$

In [25]:
x = [
   0.11, -0.06, -0.96, -0.48, -0.59, -0.42, -0.15,  1.14, 0.94, 
  -0.86, -0.08,  1.00, -2.01,  2.17, -0.20,  0.82, -0.13, 0.26, 
   0.22,  1.05
]

y = [4, 2, 4, 1, 1, 3, 4, 5, 7, 3, 5, 7, 0, 4, 2, 7, 3, 3, 2, 8]

In [26]:
## Objective function implementation
l = lambda beta: -(y[0] * (beta[0] + beta[1] * x[0]) - torch.exp(beta[0] + beta[1] * x[0]) - torch.log(torch.exp(torch.lgamma(torch.tensor([y[0] + 1],dtype = torch.double))))+
     y[1] * (beta[0] + beta[1] * x[1]) - torch.exp(beta[0] + beta[1] * x[1]) - torch.log(torch.exp(torch.lgamma(torch.tensor([y[1] + 1],dtype = torch.double))))+
     y[2] * (beta[0] + beta[1] * x[2]) - torch.exp(beta[0] + beta[1] * x[2]) - torch.log(torch.exp(torch.lgamma(torch.tensor([y[2] + 1],dtype = torch.double))))+
     y[3] * (beta[0] + beta[1] * x[3]) - torch.exp(beta[0] + beta[1] * x[3]) - torch.log(torch.exp(torch.lgamma(torch.tensor([y[3] + 1],dtype = torch.double))))+
     y[4] * (beta[0] + beta[1] * x[4]) - torch.exp(beta[0] + beta[1] * x[4]) - torch.log(torch.exp(torch.lgamma(torch.tensor([y[4] + 1],dtype = torch.double))))+
     y[5] * (beta[0] + beta[1] * x[5]) - torch.exp(beta[0] + beta[1] * x[5]) - torch.log(torch.exp(torch.lgamma(torch.tensor([y[5] + 1],dtype = torch.double))))+
     y[6] * (beta[0] + beta[1] * x[6]) - torch.exp(beta[0] + beta[1] * x[6]) - torch.log(torch.exp(torch.lgamma(torch.tensor([y[6] + 1],dtype = torch.double))))+
     y[7] * (beta[0] + beta[1] * x[7]) - torch.exp(beta[0] + beta[1] * x[7]) - torch.log(torch.exp(torch.lgamma(torch.tensor([y[7] + 1],dtype = torch.double))))+
     y[8] * (beta[0] + beta[1] * x[8]) - torch.exp(beta[0] + beta[1] * x[8]) - torch.log(torch.exp(torch.lgamma(torch.tensor([y[8] + 1],dtype = torch.double))))+
     y[9] * (beta[0] + beta[1] * x[9]) - torch.exp(beta[0] + beta[1] * x[9]) - torch.log(torch.exp(torch.lgamma(torch.tensor([y[9] + 1],dtype = torch.double)))))[0]

In [28]:
## theta 1 minimization
newton(torch.tensor([1,1],dtype = torch.double), l, tol = 1e-8, fscale = 1.0,maxit = 100, max_half = 20)

{'f': tensor(17.6547, dtype=torch.float64),
 'theta': tensor([1.2340, 0.4660], dtype=torch.float64),
 'iter': 5,
 'grad': tensor([ 3.2648e-08, -1.7604e-08], dtype=torch.float64)}

In [29]:
## theta 2 minimization
newton(torch.tensor([5,10],dtype = torch.double), l, tol = 1e-8, fscale = 1.0,maxit = 100, max_half = 20)

{'f': tensor(17.6547, dtype=torch.float64),
 'theta': tensor([1.2340, 0.4660], dtype=torch.float64),
 'iter': 19,
 'grad': tensor([6.9362e-09, 6.4082e-09], dtype=torch.float64)}

In [30]:
## theta 3 minimization
newton(torch.tensor([-1,-1],dtype = torch.double), l, tol = 1e-8,fscale = 1.0, maxit = 100, max_half = 20)

{'f': tensor(17.6547, dtype=torch.float64),
 'theta': tensor([1.2340, 0.4660], dtype=torch.float64),
 'iter': 7,
 'grad': tensor([ 2.5381e-09, -1.3687e-09], dtype=torch.float64)}